In [1]:
import pandas as pd
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
low=pd.read_csv('ecg_low.csv')
low

,Unnamed: 0,Trial 1:0back,Trial 6:0back
0,0,0.357,0.097
1,1,0.328,0.135
2,2,0.281,0.168
3,3,0.294,0.169
4,4,0.283,0.162
...,...,...,...
1689595,1689595,0.022,0.190
1689596,1689596,0.018,0.176
1689597,1689597,0.039,0.158
1689598,1689598,0.045,0.112


In [3]:
high=pd.read_csv('ecg_high.csv')

In [4]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
dfl=sc.fit_transform(low)
dfh=sc.fit_transform(high)

In [5]:
dfl=pd.DataFrame(dfl)
dfl

,0,1,2
0,-1.732050,1.509570,0.394461
1,-1.732048,1.386952,0.548937
2,-1.732046,1.188225,0.683087
3,-1.732044,1.243192,0.687153
4,-1.732042,1.196682,0.658696
...,...,...,...
1689595,1.732042,0.093116,0.772521
1689596,1.732044,0.076203,0.715609
1689597,1.732046,0.164996,0.642436
1689598,1.732048,0.190365,0.455439


In [6]:
dfh=pd.DataFrame(dfh)
dfh

,0,1,2
0,-1.732050,-0.142980,-0.366377
1,-1.732048,-0.518319,-0.346027
2,-1.732046,-0.579515,-0.337887
3,-1.732044,-0.355128,-0.325677
4,-1.732042,-0.196017,-0.317537
...,...,...,...
1689595,1.732042,-0.485680,-0.480338
1689596,1.732044,-0.514239,-0.582088
1689597,1.732046,-0.595834,-0.643138
1689598,1.732048,-0.559116,-0.753029


In [7]:
dfl=dfl.iloc[:,1:3]
dfh=dfh.iloc[:,1:3]
dfl


,1,2
0,1.509570,0.394461
1,1.386952,0.548937
2,1.188225,0.683087
3,1.243192,0.687153
4,1.196682,0.658696
...,...,...
1689595,0.093116,0.772521
1689596,0.076203,0.715609
1689597,0.164996,0.642436
1689598,0.190365,0.455439


In [8]:
from scipy.stats import skew, kurtosis 

In [9]:
import numpy as np
import pandas as pd
import neurokit2 as nk
from scipy import stats
from scipy.signal import find_peaks


features = {'mean': [],'std': [],'range': [],'slope': [],'skew': [],'kurtosis': [],'heart_rate': [],'peak_amplitude': [],'num_peaks': [],'AUC': [],'IQR': [],'CV': [], 'SDNN': [],'RMSSD': []}

sampling_rate = 256  
for i in range(dfl.shape[1]):  
    for j in range(0, len(dfl), 768):  
        set_val = dfl.iloc[j:j + 768, i]
        if len(set_val) < 768:
            continue  
        signal = set_val.values
        filtered_signal = nk.signal_filter(signal, sampling_rate=sampling_rate, lowcut=0.5, highcut=10)
        time = np.arange(len(filtered_signal)) / sampling_rate
        features['mean'].append(np.mean(filtered_signal))
        features['std'].append(np.std(filtered_signal))
        peaks, _ = find_peaks(filtered_signal, height=0)  
        features['num_peaks'].append(len(peaks))
        if len(peaks) > 1:
            peak_times = time[peaks]
            hr = 60 / np.mean(np.diff(peak_times))  
            features['heart_rate'].append(hr)

           
            rr_intervals = np.diff(peak_times)
            features['SDNN'].append(np.std(rr_intervals) if len(rr_intervals) > 1 else 0)
            features['RMSSD'].append(np.sqrt(np.mean(np.square(np.diff(rr_intervals)))) if len(rr_intervals) > 1 else 0)
        else:
            features['heart_rate'].append(0) 
            features['SDNN'].append(0)
            features['RMSSD'].append(0)

        
        if peaks.size > 0:
            features['peak_amplitude'].append(np.max(filtered_signal[peaks]))
        else:
            features['peak_amplitude'].append(0)

        
        auc = np.trapz(filtered_signal, time)
        features['AUC'].append(auc)

      
        features['range'].append(np.max(filtered_signal) - np.min(filtered_signal))
        features['slope'].append(np.polyfit(time, filtered_signal, 1)[0])  
        features['skew'].append(stats.skew(filtered_signal))
        features['kurtosis'].append(stats.kurtosis(filtered_signal))
        features['IQR'].append(np.percentile(filtered_signal, 75) - np.percentile(filtered_signal, 25))
        features['CV'].append(np.std(filtered_signal) / np.mean(filtered_signal) if np.mean(filtered_signal) != 0 else 0)


features_df = pd.DataFrame(features)

In [10]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,-0.092104,0.891441,4.337047,0.133364,0.171195,-0.548023,233.789954,1.874265,11,-0.277088,1.290298,-9.678639,0.120171,0.165441
1,-0.047414,0.883676,3.592079,0.065603,0.392042,-0.805182,203.443709,1.616383,11,-0.143302,1.309799,-18.637280,0.123505,0.210990
2,0.139554,0.949781,4.516807,-0.252800,0.637219,0.208231,292.173913,2.674505,15,0.417506,1.422394,6.805842,0.130584,0.215166
3,-0.079795,0.988857,5.254630,0.076986,-0.173193,0.158301,281.131448,1.798495,12,-0.233103,1.340614,-12.392480,0.126470,0.189439
4,0.019359,0.797367,3.565846,-0.062637,0.183347,-0.538886,237.971831,1.523067,12,0.056701,1.160179,41.188418,0.164378,0.280416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.026883,0.726054,3.528492,-0.061207,0.365612,-0.015099,229.877551,2.126367,12,0.079902,0.919683,27.008061,0.134440,0.233785
4396,-0.103005,0.794007,4.200539,0.203098,0.243042,0.121091,212.676923,2.244263,10,-0.308841,1.066425,-7.708446,0.142966,0.235617
4397,-0.006425,0.783684,3.646873,0.007095,0.382790,-0.207632,224.233577,2.205575,11,-0.018571,1.036798,-121.982547,0.120579,0.218308
4398,0.028697,0.821126,3.876250,-0.020954,0.398142,-0.300458,258.151261,2.453767,13,0.086671,1.156980,28.613224,0.100258,0.165087


In [11]:
import numpy as np
import pandas as pd
import neurokit2 as nk
from scipy import stats
from scipy.signal import find_peaks


features = {'mean': [],'std': [],'range': [],'slope': [],'skew': [],'kurtosis': [],'heart_rate': [],'peak_amplitude': [],'num_peaks': [],'AUC': [],'IQR': [],'CV': [], 'SDNN': [],'RMSSD': []}


sampling_rate = 256 

for i in range(dfh.shape[1]):  
    for j in range(0, len(dfh), 768):  
        set_val = dfh.iloc[j:j + 768, i]
        if len(set_val) < 768:
            continue  

       
        signal = set_val.values

       
        filtered_signal = nk.signal_filter(signal, sampling_rate=sampling_rate, lowcut=0.5, highcut=10)

        
        time = np.arange(len(filtered_signal)) / sampling_rate
        
       
        features['mean'].append(np.mean(filtered_signal))
        features['std'].append(np.std(filtered_signal))

       
        peaks, _ = find_peaks(filtered_signal, height=0)  
        features['num_peaks'].append(len(peaks))
        
       
        if len(peaks) > 1:
            peak_times = time[peaks]
            hr = 60 / np.mean(np.diff(peak_times))  # In beats per minute
            features['heart_rate'].append(hr)

           
            rr_intervals = np.diff(peak_times)
            features['SDNN'].append(np.std(rr_intervals) if len(rr_intervals) > 1 else 0)
            features['RMSSD'].append(np.sqrt(np.mean(np.square(np.diff(rr_intervals)))) if len(rr_intervals) > 1 else 0)
        else:
            features['heart_rate'].append(0) 
            features['SDNN'].append(0)
            features['RMSSD'].append(0)

        # Peak Amplitude
        if peaks.size > 0:
            features['peak_amplitude'].append(np.max(filtered_signal[peaks]))
        else:
            features['peak_amplitude'].append(0)

       
        auc = np.trapz(filtered_signal, time)
        features['AUC'].append(auc)

        # Range, slope, skew and kurtosis
        features['range'].append(np.max(filtered_signal) - np.min(filtered_signal))
        features['slope'].append(np.polyfit(time, filtered_signal, 1)[0])  
        features['skew'].append(stats.skew(filtered_signal))
        features['kurtosis'].append(stats.kurtosis(filtered_signal))
        features['IQR'].append(np.percentile(filtered_signal, 75) - np.percentile(filtered_signal, 25))
        features['CV'].append(np.std(filtered_signal) / np.mean(filtered_signal) if np.mean(filtered_signal) != 0 else 0)
features_dfh = pd.DataFrame(features)

In [12]:
features_dfh

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.082729,0.944140,4.063316,-0.107102,0.439223,-0.757066,235.648536,2.204694,12,0.245847,1.476180,11.412441,0.111309,0.200330
1,0.070160,0.816153,3.550817,-0.131023,0.378033,-0.676004,247.741935,1.969181,13,0.209917,1.161709,11.632708,0.125173,0.206964
2,0.017936,0.833094,3.531574,-0.025122,0.321377,-0.762765,254.585635,1.689894,13,0.054423,1.268504,46.448094,0.086576,0.152894
3,0.112320,0.950747,4.573365,-0.282879,0.468533,-0.487690,270.769231,2.306298,12,0.334656,1.508181,8.464652,0.083483,0.141679
4,-0.007757,0.820609,3.600586,-0.020276,0.318037,-0.732794,250.434783,1.618997,13,-0.025017,1.182011,-105.795587,0.129775,0.164716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.010076,0.720335,3.938874,-0.062258,0.490657,-0.149830,243.458213,2.172181,12,0.029506,1.005068,71.489518,0.103731,0.182397
4396,-0.085172,0.774212,4.020507,0.144065,0.271789,0.107785,251.803279,2.092035,11,-0.255580,0.974473,-9.089956,0.108478,0.187920
4397,-0.061574,0.824456,3.648631,0.176182,0.519533,-0.276543,226.791946,2.062993,12,-0.186780,1.122846,-13.389667,0.113716,0.194667
4398,-0.009903,0.764018,3.571977,0.074637,0.455318,-0.330244,257.168950,2.151009,12,-0.029240,1.035980,-77.151156,0.097494,0.170009


In [13]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,-0.092104,0.891441,4.337047,0.133364,0.171195,-0.548023,233.789954,1.874265,11,-0.277088,1.290298,-9.678639,0.120171,0.165441
1,-0.047414,0.883676,3.592079,0.065603,0.392042,-0.805182,203.443709,1.616383,11,-0.143302,1.309799,-18.637280,0.123505,0.210990
2,0.139554,0.949781,4.516807,-0.252800,0.637219,0.208231,292.173913,2.674505,15,0.417506,1.422394,6.805842,0.130584,0.215166
3,-0.079795,0.988857,5.254630,0.076986,-0.173193,0.158301,281.131448,1.798495,12,-0.233103,1.340614,-12.392480,0.126470,0.189439
4,0.019359,0.797367,3.565846,-0.062637,0.183347,-0.538886,237.971831,1.523067,12,0.056701,1.160179,41.188418,0.164378,0.280416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.026883,0.726054,3.528492,-0.061207,0.365612,-0.015099,229.877551,2.126367,12,0.079902,0.919683,27.008061,0.134440,0.233785
4396,-0.103005,0.794007,4.200539,0.203098,0.243042,0.121091,212.676923,2.244263,10,-0.308841,1.066425,-7.708446,0.142966,0.235617
4397,-0.006425,0.783684,3.646873,0.007095,0.382790,-0.207632,224.233577,2.205575,11,-0.018571,1.036798,-121.982547,0.120579,0.218308
4398,0.028697,0.821126,3.876250,-0.020954,0.398142,-0.300458,258.151261,2.453767,13,0.086671,1.156980,28.613224,0.100258,0.165087


In [14]:
features_dfh

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.082729,0.944140,4.063316,-0.107102,0.439223,-0.757066,235.648536,2.204694,12,0.245847,1.476180,11.412441,0.111309,0.200330
1,0.070160,0.816153,3.550817,-0.131023,0.378033,-0.676004,247.741935,1.969181,13,0.209917,1.161709,11.632708,0.125173,0.206964
2,0.017936,0.833094,3.531574,-0.025122,0.321377,-0.762765,254.585635,1.689894,13,0.054423,1.268504,46.448094,0.086576,0.152894
3,0.112320,0.950747,4.573365,-0.282879,0.468533,-0.487690,270.769231,2.306298,12,0.334656,1.508181,8.464652,0.083483,0.141679
4,-0.007757,0.820609,3.600586,-0.020276,0.318037,-0.732794,250.434783,1.618997,13,-0.025017,1.182011,-105.795587,0.129775,0.164716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.010076,0.720335,3.938874,-0.062258,0.490657,-0.149830,243.458213,2.172181,12,0.029506,1.005068,71.489518,0.103731,0.182397
4396,-0.085172,0.774212,4.020507,0.144065,0.271789,0.107785,251.803279,2.092035,11,-0.255580,0.974473,-9.089956,0.108478,0.187920
4397,-0.061574,0.824456,3.648631,0.176182,0.519533,-0.276543,226.791946,2.062993,12,-0.186780,1.122846,-13.389667,0.113716,0.194667
4398,-0.009903,0.764018,3.571977,0.074637,0.455318,-0.330244,257.168950,2.151009,12,-0.029240,1.035980,-77.151156,0.097494,0.170009


In [15]:
import pywt

In [16]:

import pywt
fs=256
def extract_features(signal):
    sampling_freq = 256
    wavelet = 'db4'
    coeffs = pywt.wavedec(signal, wavelet, level=5)
    cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
    vlf_power = np.sum(np.square(cA5))
    lf_power = np.sum(np.square(cD5))
    hf_power = np.sum(np.square(cD4))
    total_power = vlf_power + lf_power + hf_power
    lf_hf_ratio = lf_power / hf_power

    return {
        'VLF': vlf_power,
        'LF': lf_power,
        'HF': hf_power,
        'total_power': total_power,
        'lf/hf':lf_hf_ratio
         }

# Process data in chunks of 768 samples
results = []
i,j,k=0,1,0
while k<2:
    while j<2201:
        while i<768*j:
            set=dfl.iloc[i:i+768,k]
            ff=extract_features(set.values)
            i=i+768
            results.append(ff)
        j=j+1
    k=k+1
    j=0
    i=0
    


In [17]:
rs=pd.DataFrame(results)

In [18]:
rs

,VLF,LF,HF,total_power,lf/hf
0,687.258336,115.164735,411.545886,1213.968957,0.279834
1,945.662142,123.161847,221.983601,1290.807590,0.554824
2,3113.331880,139.537410,310.321925,3563.191215,0.449654
3,677.470920,239.033040,432.482386,1348.986346,0.552700
4,466.706736,88.542834,406.610926,961.860495,0.217758
...,...,...,...,...,...
4395,236.895225,251.851686,291.719605,780.466516,0.863335
4396,247.179383,136.499461,266.899375,650.578219,0.511427
4397,257.753081,210.708023,380.296755,848.757859,0.554062
4398,316.428319,215.989716,401.313961,933.731996,0.538206


In [19]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,-0.092104,0.891441,4.337047,0.133364,0.171195,-0.548023,233.789954,1.874265,11,-0.277088,1.290298,-9.678639,0.120171,0.165441
1,-0.047414,0.883676,3.592079,0.065603,0.392042,-0.805182,203.443709,1.616383,11,-0.143302,1.309799,-18.637280,0.123505,0.210990
2,0.139554,0.949781,4.516807,-0.252800,0.637219,0.208231,292.173913,2.674505,15,0.417506,1.422394,6.805842,0.130584,0.215166
3,-0.079795,0.988857,5.254630,0.076986,-0.173193,0.158301,281.131448,1.798495,12,-0.233103,1.340614,-12.392480,0.126470,0.189439
4,0.019359,0.797367,3.565846,-0.062637,0.183347,-0.538886,237.971831,1.523067,12,0.056701,1.160179,41.188418,0.164378,0.280416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.026883,0.726054,3.528492,-0.061207,0.365612,-0.015099,229.877551,2.126367,12,0.079902,0.919683,27.008061,0.134440,0.233785
4396,-0.103005,0.794007,4.200539,0.203098,0.243042,0.121091,212.676923,2.244263,10,-0.308841,1.066425,-7.708446,0.142966,0.235617
4397,-0.006425,0.783684,3.646873,0.007095,0.382790,-0.207632,224.233577,2.205575,11,-0.018571,1.036798,-121.982547,0.120579,0.218308
4398,0.028697,0.821126,3.876250,-0.020954,0.398142,-0.300458,258.151261,2.453767,13,0.086671,1.156980,28.613224,0.100258,0.165087


In [20]:
low_data=pd.concat([features_df,rs],axis=1)

In [21]:
low_data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,-0.092104,0.891441,4.337047,0.133364,0.171195,-0.548023,233.789954,1.874265,11,-0.277088,1.290298,-9.678639,0.120171,0.165441,687.258336,115.164735,411.545886,1213.968957,0.279834
1,-0.047414,0.883676,3.592079,0.065603,0.392042,-0.805182,203.443709,1.616383,11,-0.143302,1.309799,-18.637280,0.123505,0.210990,945.662142,123.161847,221.983601,1290.807590,0.554824
2,0.139554,0.949781,4.516807,-0.252800,0.637219,0.208231,292.173913,2.674505,15,0.417506,1.422394,6.805842,0.130584,0.215166,3113.331880,139.537410,310.321925,3563.191215,0.449654
3,-0.079795,0.988857,5.254630,0.076986,-0.173193,0.158301,281.131448,1.798495,12,-0.233103,1.340614,-12.392480,0.126470,0.189439,677.470920,239.033040,432.482386,1348.986346,0.552700
4,0.019359,0.797367,3.565846,-0.062637,0.183347,-0.538886,237.971831,1.523067,12,0.056701,1.160179,41.188418,0.164378,0.280416,466.706736,88.542834,406.610926,961.860495,0.217758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.026883,0.726054,3.528492,-0.061207,0.365612,-0.015099,229.877551,2.126367,12,0.079902,0.919683,27.008061,0.134440,0.233785,236.895225,251.851686,291.719605,780.466516,0.863335
4396,-0.103005,0.794007,4.200539,0.203098,0.243042,0.121091,212.676923,2.244263,10,-0.308841,1.066425,-7.708446,0.142966,0.235617,247.179383,136.499461,266.899375,650.578219,0.511427
4397,-0.006425,0.783684,3.646873,0.007095,0.382790,-0.207632,224.233577,2.205575,11,-0.018571,1.036798,-121.982547,0.120579,0.218308,257.753081,210.708023,380.296755,848.757859,0.554062
4398,0.028697,0.821126,3.876250,-0.020954,0.398142,-0.300458,258.151261,2.453767,13,0.086671,1.156980,28.613224,0.100258,0.165087,316.428319,215.989716,401.313961,933.731996,0.538206


In [22]:
low

,Unnamed: 0,Trial 1:0back,Trial 6:0back
0,0,0.357,0.097
1,1,0.328,0.135
2,2,0.281,0.168
3,3,0.294,0.169
4,4,0.283,0.162
...,...,...,...
1689595,1689595,0.022,0.190
1689596,1689596,0.018,0.176
1689597,1689597,0.039,0.158
1689598,1689598,0.045,0.112


In [23]:
fs=256
def extract_features(signal):
    sampling_freq = 256
    wavelet = 'db4'
    coeffs = pywt.wavedec(signal, wavelet, level=5)
    cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
    vlf_power = np.sum(np.square(cA5))
    lf_power = np.sum(np.square(cD5))
    hf_power = np.sum(np.square(cD4))
    total_power = vlf_power + lf_power + hf_power
    lf_hf_ratio = lf_power / hf_power

    return {
        'VLF': vlf_power,
        'LF': lf_power,
        'HF': hf_power,
        'total_power': total_power,
        'lf/hf':lf_hf_ratio
         }

# Process data in chunks of 768 samples
results = []
i,j,k=0,1,0
while k<2:
    while j<2201:
        while i<768*j:
            set=dfh.iloc[i:i+768,k]
            ff=extract_features(set.values)
            i=i+768
            results.append(ff)
        j=j+1
    k=k+1
    j=0
    i=0
    

In [24]:
rsh=pd.DataFrame(results)

In [25]:
rsh

,VLF,LF,HF,total_power,lf/hf
0,474.473357,153.097910,290.101702,917.672969,0.527739
1,571.324106,76.880892,220.364277,868.569274,0.348881
2,391.441831,162.941625,395.150148,949.533604,0.412354
3,423.689081,144.492240,507.549134,1075.730455,0.284686
4,600.915177,145.030087,393.331584,1139.276848,0.368722
...,...,...,...,...,...
4395,267.859674,276.107622,221.424383,765.391679,1.246961
4396,421.565277,234.621428,365.482071,1021.668776,0.641951
4397,338.501818,172.003046,370.229282,880.734146,0.464585
4398,340.961467,132.620510,350.062469,823.644446,0.378848


In [26]:
high_data=pd.concat([features_dfh,rsh],axis=1)

In [27]:
high_data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,0.082729,0.944140,4.063316,-0.107102,0.439223,-0.757066,235.648536,2.204694,12,0.245847,1.476180,11.412441,0.111309,0.200330,474.473357,153.097910,290.101702,917.672969,0.527739
1,0.070160,0.816153,3.550817,-0.131023,0.378033,-0.676004,247.741935,1.969181,13,0.209917,1.161709,11.632708,0.125173,0.206964,571.324106,76.880892,220.364277,868.569274,0.348881
2,0.017936,0.833094,3.531574,-0.025122,0.321377,-0.762765,254.585635,1.689894,13,0.054423,1.268504,46.448094,0.086576,0.152894,391.441831,162.941625,395.150148,949.533604,0.412354
3,0.112320,0.950747,4.573365,-0.282879,0.468533,-0.487690,270.769231,2.306298,12,0.334656,1.508181,8.464652,0.083483,0.141679,423.689081,144.492240,507.549134,1075.730455,0.284686
4,-0.007757,0.820609,3.600586,-0.020276,0.318037,-0.732794,250.434783,1.618997,13,-0.025017,1.182011,-105.795587,0.129775,0.164716,600.915177,145.030087,393.331584,1139.276848,0.368722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.010076,0.720335,3.938874,-0.062258,0.490657,-0.149830,243.458213,2.172181,12,0.029506,1.005068,71.489518,0.103731,0.182397,267.859674,276.107622,221.424383,765.391679,1.246961
4396,-0.085172,0.774212,4.020507,0.144065,0.271789,0.107785,251.803279,2.092035,11,-0.255580,0.974473,-9.089956,0.108478,0.187920,421.565277,234.621428,365.482071,1021.668776,0.641951
4397,-0.061574,0.824456,3.648631,0.176182,0.519533,-0.276543,226.791946,2.062993,12,-0.186780,1.122846,-13.389667,0.113716,0.194667,338.501818,172.003046,370.229282,880.734146,0.464585
4398,-0.009903,0.764018,3.571977,0.074637,0.455318,-0.330244,257.168950,2.151009,12,-0.029240,1.035980,-77.151156,0.097494,0.170009,340.961467,132.620510,350.062469,823.644446,0.378848


In [28]:
low_data['label']=0
high_data['label']=1

In [29]:
data=pd.concat([low_data,high_data],ignore_index=True)
data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf,label
0,-0.092104,0.891441,4.337047,0.133364,0.171195,-0.548023,233.789954,1.874265,11,-0.277088,1.290298,-9.678639,0.120171,0.165441,687.258336,115.164735,411.545886,1213.968957,0.279834,0
1,-0.047414,0.883676,3.592079,0.065603,0.392042,-0.805182,203.443709,1.616383,11,-0.143302,1.309799,-18.637280,0.123505,0.210990,945.662142,123.161847,221.983601,1290.807590,0.554824,0
2,0.139554,0.949781,4.516807,-0.252800,0.637219,0.208231,292.173913,2.674505,15,0.417506,1.422394,6.805842,0.130584,0.215166,3113.331880,139.537410,310.321925,3563.191215,0.449654,0
3,-0.079795,0.988857,5.254630,0.076986,-0.173193,0.158301,281.131448,1.798495,12,-0.233103,1.340614,-12.392480,0.126470,0.189439,677.470920,239.033040,432.482386,1348.986346,0.552700,0
4,0.019359,0.797367,3.565846,-0.062637,0.183347,-0.538886,237.971831,1.523067,12,0.056701,1.160179,41.188418,0.164378,0.280416,466.706736,88.542834,406.610926,961.860495,0.217758,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,0.010076,0.720335,3.938874,-0.062258,0.490657,-0.149830,243.458213,2.172181,12,0.029506,1.005068,71.489518,0.103731,0.182397,267.859674,276.107622,221.424383,765.391679,1.246961,1
8796,-0.085172,0.774212,4.020507,0.144065,0.271789,0.107785,251.803279,2.092035,11,-0.255580,0.974473,-9.089956,0.108478,0.187920,421.565277,234.621428,365.482071,1021.668776,0.641951,1
8797,-0.061574,0.824456,3.648631,0.176182,0.519533,-0.276543,226.791946,2.062993,12,-0.186780,1.122846,-13.389667,0.113716,0.194667,338.501818,172.003046,370.229282,880.734146,0.464585,1
8798,-0.009903,0.764018,3.571977,0.074637,0.455318,-0.330244,257.168950,2.151009,12,-0.029240,1.035980,-77.151156,0.097494,0.170009,340.961467,132.620510,350.062469,823.644446,0.378848,1


In [80]:
data.to_csv('ecg_feature.csv')

In [30]:
x=data.iloc[:,0:19]

In [31]:
x

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,-0.092104,0.891441,4.337047,0.133364,0.171195,-0.548023,233.789954,1.874265,11,-0.277088,1.290298,-9.678639,0.120171,0.165441,687.258336,115.164735,411.545886,1213.968957,0.279834
1,-0.047414,0.883676,3.592079,0.065603,0.392042,-0.805182,203.443709,1.616383,11,-0.143302,1.309799,-18.637280,0.123505,0.210990,945.662142,123.161847,221.983601,1290.807590,0.554824
2,0.139554,0.949781,4.516807,-0.252800,0.637219,0.208231,292.173913,2.674505,15,0.417506,1.422394,6.805842,0.130584,0.215166,3113.331880,139.537410,310.321925,3563.191215,0.449654
3,-0.079795,0.988857,5.254630,0.076986,-0.173193,0.158301,281.131448,1.798495,12,-0.233103,1.340614,-12.392480,0.126470,0.189439,677.470920,239.033040,432.482386,1348.986346,0.552700
4,0.019359,0.797367,3.565846,-0.062637,0.183347,-0.538886,237.971831,1.523067,12,0.056701,1.160179,41.188418,0.164378,0.280416,466.706736,88.542834,406.610926,961.860495,0.217758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,0.010076,0.720335,3.938874,-0.062258,0.490657,-0.149830,243.458213,2.172181,12,0.029506,1.005068,71.489518,0.103731,0.182397,267.859674,276.107622,221.424383,765.391679,1.246961
8796,-0.085172,0.774212,4.020507,0.144065,0.271789,0.107785,251.803279,2.092035,11,-0.255580,0.974473,-9.089956,0.108478,0.187920,421.565277,234.621428,365.482071,1021.668776,0.641951
8797,-0.061574,0.824456,3.648631,0.176182,0.519533,-0.276543,226.791946,2.062993,12,-0.186780,1.122846,-13.389667,0.113716,0.194667,338.501818,172.003046,370.229282,880.734146,0.464585
8798,-0.009903,0.764018,3.571977,0.074637,0.455318,-0.330244,257.168950,2.151009,12,-0.029240,1.035980,-77.151156,0.097494,0.170009,340.961467,132.620510,350.062469,823.644446,0.378848


In [32]:
y=data.iloc[:,19]

In [33]:
y

0       0
1       0
2       0
3       0
4       0
       ..
8795    1
8796    1
8797    1
8798    1
8799    1
Name: label, Length: 8800, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [35]:
from sklearn.ensemble import RandomForestClassifier


In [36]:
rfc=RandomForestClassifier(n_estimators=1000)


In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 1000)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [38]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred)*100)

Accuracy :  70.87121212121212
